In [1]:
import os


In [2]:
%pwd


'c:\\Users\\Hamza\\Desktop\\DL-kidney-disease-classification\\research'

In [3]:
os.chdir("..")


In [4]:
%pwd

'c:\\Users\\Hamza\\Desktop\\DL-kidney-disease-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path
import mlflow
from mlflow.tracking import MlflowClient
import dagshub

In [6]:
@dataclass(frozen=True)
class MLflowConfig:
    repo_owner: str
    repo_name: str
    mlflow_uri: str
    model_name: str

In [7]:
class MLflowConfigManager:
    def __init__(
        self,
        repo_owner: str = "hamzaboughanim06",
        repo_name: str = "DL-kidney-disease-classification"
    ):
        self.repo_owner = repo_owner
        self.repo_name = repo_name
        self.mlflow_uri = f"https://dagshub.com/{repo_owner}/{repo_name}.mlflow"
    
    def get_mlflow_config(self, model_name: str) -> MLflowConfig:
        return MLflowConfig(
            repo_owner=self.repo_owner,
            repo_name=self.repo_name,
            mlflow_uri=self.mlflow_uri,
            model_name=model_name
        )

In [8]:
class ModelStatusManager:
    def __init__(self, config: MLflowConfig):
        self.config = config
        self.client = None
    
    def initialize_connection(self):
        """Initialize DagHub and MLflow connection"""
        dagshub.init(
            repo_owner=self.config.repo_owner,
            repo_name=self.config.repo_name,
            mlflow=True
        )
        
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        
        self.client = MlflowClient()
        print(f"✅ Connected to: {self.config.mlflow_uri}")
    
    def view_model_versions(self):
        """View all versions of the model"""
        versions = self.client.search_model_versions(f"name='{self.config.model_name}'")
        versions = sorted(versions, key=lambda x: int(x.version), reverse=True)
        
        print(f"\n📊 Model: {self.config.model_name}")
        print("-" * 60)
        
        for v in versions:
            emoji = {"None": "⚪", "Staging": "🟡", "Production": "🟢", "Archived": "⚫"}
            print(f"{emoji.get(v.current_stage, '⚪')} Version {v.version} | Stage: {v.current_stage} | Status: {v.status}")
        
        return versions
    
    def change_status(self, version: int, new_stage: str, archive_existing: bool = True):
        """Change model version stage"""
        print(f"\n🔄 Changing status:")
        print(f"   Model: {self.config.model_name}")
        print(f"   Version: {version}")
        print(f"   New Stage: {new_stage}")
        
        self.client.transition_model_version_stage(
            name=self.config.model_name,
            version=version,
            stage=new_stage,
            archive_existing_versions=archive_existing
        )
        
        print(f"✅ Success! Version {version} is now in '{new_stage}' stage")
        
        # Show updated versions
        self.view_model_versions()

In [10]:
try:
    # Step 1: Get configuration
    config_manager = MLflowConfigManager()
    mlflow_config = config_manager.get_mlflow_config(model_name="VGG16Model")
    
    # Step 2: Initialize status manager
    status_manager = ModelStatusManager(config=mlflow_config)
    status_manager.initialize_connection()
    
    # View current versions
    status_manager.view_model_versions()
    
    
    VERSION = 8        
    NEW_STAGE = "Production"
    
    status_manager.change_status(
        version=VERSION,
        new_stage=NEW_STAGE,
        archive_existing=True
    )
    
except Exception as e:
    raise e

Initialized MLflow to track repo "hamzaboughanim06/DL-kidney-disease-classification"

Repository hamzaboughanim06/DL-kidney-disease-classification initialized!

✅ Connected to: https://dagshub.com/hamzaboughanim06/DL-kidney-disease-classification.mlflow

📊 Model: VGG16Model
------------------------------------------------------------
⚪ Version 8 | Stage: None | Status: READY
⚪ Version 7 | Stage: None | Status: READY
⚪ Version 6 | Stage: None | Status: READY
⚪ Version 4 | Stage: None | Status: READY
⚪ Version 3 | Stage: None | Status: READY
⚪ Version 2 | Stage: None | Status: READY
⚪ Version 1 | Stage: None | Status: READY

🔄 Changing status:
   Model: VGG16Model
   Version: 8
   New Stage: Production
✅ Success! Version 8 is now in 'Production' stage

📊 Model: VGG16Model
------------------------------------------------------------
🟢 Version 8 | Stage: Production | Status: READY
⚪ Version 7 | Stage: None | Status: READY
⚪ Version 6 | Stage: None | Status: READY
⚪ Version 4 | Stage: None | Status: READY
⚪ Version 3 | Stage: None | Status: READY
⚪ Version 2 | Stage: None | Status: READY
⚪ Version 1 | Stage: None | Status: READY
